# 83. NumPyスクラッチ実装（高速化編）

**Unit 0.4: 空間の帰納バイアス - CNN & 局所性の科学**

---

## このノートブックの位置づけ

前回の愚直な実装は動作しましたが、非常に遅いものでした。今回は、NumPyのベクトル化機能を活用して**劇的に高速化**する方法を学びます。

```
Unit 0.4 学習マップ
==================

Section A: 畳み込みの世界への入口
├── 80. 畳み込みとは何か（直感編）  ✅ 完了
├── 81. 畳み込みの数学的定義  ✅ 完了
├── 82. NumPyスクラッチ実装（基礎編）  ✅ 完了
├── 83. NumPyスクラッチ実装（高速化編）  ← 今ここ
├── 84. 古典的フィルタの解剖学
└── 85. カーネルの可視化と解釈
```

## 学習目標

このノートブックを終えると、以下のことができるようになります：

1. **`sliding_window_view` を使った高速畳み込みを実装できる**
2. **im2col テクニックを理解し実装できる**
3. **畳み込みが行列積に変換できることを理解できる**
4. **愚直な実装との速度差を測定・比較できる**
5. **PyTorchの内部実装との関係を説明できる**

## 目次

1. [前回の振り返りと問題提起](#1-前回の振り返りと問題提起)
2. [sliding_window_view の魔法](#2-sliding_window_view-の魔法)
3. [ベクトル化による高速畳み込み](#3-ベクトル化による高速畳み込み)
4. [im2col テクニック](#4-im2col-テクニック)
5. [PyTorchの内部実装との比較](#5-pytorchの内部実装との比較)
6. [ベンチマーク：3つの実装の速度比較](#6-ベンチマーク3つの実装の速度比較)

---

## 環境セットアップ

In [ ]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# PyTorchのインポート（比較用）
try:
    import torch
    import torch.nn.functional as F
    TORCH_AVAILABLE = True
    print(f"PyTorch version: {torch.__version__}")
except ImportError:
    TORCH_AVAILABLE = False
    print("PyTorchは利用できません（比較はスキップされます）")

# 日本語フォント設定
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)
print("環境準備完了")

---

## 1. 前回の振り返りと問題提起

前回実装した愚直なバージョンを再度定義し、問題点を確認します。

In [ ]:
# 前回の愚直な実装

def conv2d_naive(image, kernel, padding=0, stride=1):
    """
    愚直な2D畳み込み実装（forループ版）
    """
    # パディング
    if padding > 0:
        image = np.pad(image, padding, mode='constant', constant_values=0)
    
    img_h, img_w = image.shape
    k_h, k_w = kernel.shape
    
    out_h = (img_h - k_h) // stride + 1
    out_w = (img_w - k_w) // stride + 1
    
    output = np.zeros((out_h, out_w))
    
    for i in range(out_h):
        for j in range(out_w):
            row = i * stride
            col = j * stride
            window = image[row:row+k_h, col:col+k_w]
            output[i, j] = np.sum(window * kernel)
    
    return output

# 問題の確認
test_size = 128
test_img = np.random.rand(test_size, test_size)
test_kernel = np.random.rand(3, 3)

start = time.time()
result_naive = conv2d_naive(test_img, test_kernel, padding=1)
naive_time = time.time() - start

print(f"愚直な実装（{test_size}×{test_size}画像、3×3カーネル）:")
print(f"  計算時間: {naive_time*1000:.2f} ms")
print(f"\nこれを高速化していきます！")

---

## 2. sliding_window_view の魔法

`numpy.lib.stride_tricks.sliding_window_view` は、配列のスライディングウィンドウを**コピーなし**で生成する強力な関数です。

### 2.1 sliding_window_view の基本

In [ ]:
# 1次元での例

arr_1d = np.arange(10)
print("元の配列:")
print(arr_1d)

# ウィンドウサイズ3でスライディングビュー
windows = sliding_window_view(arr_1d, window_shape=3)
print(f"\nスライディングウィンドウ（サイズ3）:")
print(f"形状: {windows.shape}")
print(windows)

In [ ]:
# 2次元での例

arr_2d = np.arange(16).reshape(4, 4)
print("元の配列 (4×4):")
print(arr_2d)

# 2×2のウィンドウでスライディングビュー
windows_2d = sliding_window_view(arr_2d, window_shape=(2, 2))
print(f"\nスライディングウィンドウ（2×2）:")
print(f"形状: {windows_2d.shape}")
print("  → (出力高さ, 出力幅, カーネル高さ, カーネル幅)")

print(f"\n位置(0,0)のウィンドウ:")
print(windows_2d[0, 0])

print(f"\n位置(1,1)のウィンドウ:")
print(windows_2d[1, 1])

print(f"\n位置(2,2)のウィンドウ:")
print(windows_2d[2, 2])

### 2.2 メモリビュー（コピーなし）の概念

`sliding_window_view` の最大の利点は、**データをコピーしない**ことです。元の配列のメモリを「異なる視点」で見ているだけです。

In [ ]:
# メモリ共有の確認

original = np.arange(10)
view = sliding_window_view(original, window_shape=3)

print("元の配列:", original)
print("ウィンドウビュー:")
print(view)

# 元の配列を変更
original[5] = 999

print("\n元の配列[5]を999に変更後:")
print("元の配列:", original)
print("ウィンドウビュー:")
print(view)

print("\n→ ウィンドウビューも自動的に変化！（メモリを共有している）")

In [ ]:
# メモリ使用量の比較

large_arr = np.random.rand(1000, 1000)

# コピーを作る場合
def make_copies(arr, k_size):
    """全てのウィンドウをコピーして保存"""
    h, w = arr.shape
    out_h, out_w = h - k_size + 1, w - k_size + 1
    copies = np.zeros((out_h, out_w, k_size, k_size))
    for i in range(out_h):
        for j in range(out_w):
            copies[i, j] = arr[i:i+k_size, j:j+k_size].copy()
    return copies

print(f"元の配列: {large_arr.nbytes / 1e6:.1f} MB")

# sliding_window_view（コピーなし）
view = sliding_window_view(large_arr, window_shape=(3, 3))
print(f"sliding_window_view: {view.nbytes / 1e6:.1f} MB（見かけ上）")
print(f"  実際のメモリ増加: ほぼ0（ビューのためメタデータのみ）")

# 参考：全コピーした場合
expected_copy_size = (1000-2) * (1000-2) * 3 * 3 * 8  # float64 = 8 bytes
print(f"\n全コピーした場合: {expected_copy_size / 1e6:.1f} MB")

### 2.3 4次元配列としての画像パッチ

2D畳み込みでは、`sliding_window_view` は4次元配列を返します：

```
(出力高さ, 出力幅, カーネル高さ, カーネル幅)
```

これにより、全てのウィンドウに対して一度に演算を行えます。

In [ ]:
# 4次元配列の構造を可視化

fig, axes = plt.subplots(2, 3, figsize=(14, 9))

# サンプル画像
sample_img = np.arange(25).reshape(5, 5)
kernel_size = (3, 3)
windows = sliding_window_view(sample_img, window_shape=kernel_size)

# 元画像
ax = axes[0, 0]
ax.imshow(sample_img, cmap='Blues')
for i in range(5):
    for j in range(5):
        ax.text(j, i, f'{sample_img[i,j]}', ha='center', va='center', fontsize=10)
ax.set_title(f'元画像 (5×5)', fontsize=11)
ax.axis('off')

# 各位置のウィンドウを表示
positions = [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1)]
for ax, (i, j) in zip(axes.flatten()[1:], positions):
    window = windows[i, j]
    ax.imshow(window, cmap='Oranges', vmin=0, vmax=24)
    for ii in range(3):
        for jj in range(3):
            ax.text(jj, ii, f'{window[ii,jj]}', ha='center', va='center', fontsize=11)
    ax.set_title(f'windows[{i}, {j}]', fontsize=11)
    ax.axis('off')

plt.suptitle(f'sliding_window_view の出力\n形状: {windows.shape} = (出力H, 出力W, カーネルH, カーネルW)', 
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nwindows.shape = {windows.shape}")
print(f"  - windows[i, j] で位置(i,j)のカーネルサイズのウィンドウを取得")
print(f"  - 全ての位置のウィンドウが一度に取得できる！")

---

## 3. ベクトル化による高速畳み込み

`sliding_window_view` を使えば、forループなしで畳み込みを実装できます。

In [ ]:
def conv2d_vectorized(image, kernel, padding=0, stride=1):
    """
    sliding_window_view を使った高速2D畳み込み
    
    Parameters:
    -----------
    image : ndarray, shape (H, W)
        入力画像
    kernel : ndarray, shape (kH, kW)
        カーネル
    padding : int
        パディング幅
    stride : int
        ストライド
    
    Returns:
    --------
    output : ndarray
    """
    # パディング
    if padding > 0:
        image = np.pad(image, padding, mode='constant', constant_values=0)
    
    k_h, k_w = kernel.shape
    
    # 全てのウィンドウを一度に取得
    windows = sliding_window_view(image, window_shape=(k_h, k_w))
    # shape: (出力H, 出力W, k_h, k_w)
    
    # ストライド適用
    if stride > 1:
        windows = windows[::stride, ::stride]
    
    # ベクトル化された畳み込み計算
    # windows * kernel: ブロードキャストで全位置に適用
    # sum(axis=(-2, -1)): 最後の2軸（カーネル次元）で合計
    output = np.sum(windows * kernel, axis=(-2, -1))
    
    return output

In [ ]:
# 計算過程の詳細説明

# 小さな例で確認
small_img = np.arange(16).reshape(4, 4).astype(float)
small_kernel = np.array([[1, 0], [0, 1]], dtype=float)  # 対角成分の和

print("入力画像 (4×4):")
print(small_img)
print("\nカーネル (2×2):")
print(small_kernel)

# ステップ1: ウィンドウを取得
windows = sliding_window_view(small_img, window_shape=(2, 2))
print(f"\nステップ1: sliding_window_view")
print(f"  出力形状: {windows.shape}")

# ステップ2: カーネルとの要素積
products = windows * small_kernel
print(f"\nステップ2: windows * kernel（ブロードキャスト）")
print(f"  出力形状: {products.shape}")
print(f"  位置(0,0)の要素積:")
print(products[0, 0])

# ステップ3: 最後の2軸で合計
output = np.sum(products, axis=(-2, -1))
print(f"\nステップ3: sum(axis=(-2,-1))")
print(f"  出力形状: {output.shape}")
print(f"  結果:")
print(output)

# 検証（愚直版と比較）
output_naive = conv2d_naive(small_img, small_kernel)
print(f"\n検証（愚直版との差）: {np.max(np.abs(output - output_naive))}")

In [ ]:
# 速度比較

test_img = np.random.rand(128, 128)
test_kernel = np.random.rand(3, 3)

# 愚直版
start = time.time()
for _ in range(10):
    result_naive = conv2d_naive(test_img, test_kernel, padding=1)
naive_time = (time.time() - start) / 10

# ベクトル化版
start = time.time()
for _ in range(10):
    result_vec = conv2d_vectorized(test_img, test_kernel, padding=1)
vec_time = (time.time() - start) / 10

print(f"128×128画像、3×3カーネル:")
print(f"  愚直版:      {naive_time*1000:.2f} ms")
print(f"  ベクトル化版: {vec_time*1000:.2f} ms")
print(f"  高速化率:    {naive_time/vec_time:.1f}x")

# 結果が一致することを確認
print(f"\n結果の最大差: {np.max(np.abs(result_naive - result_vec)):.2e}")

---

## 4. im2col テクニック

**im2col (image to column)** は、畳み込みを**行列積**に変換するテクニックです。GPUでの実装で広く使われています。

### 4.1 im2col の概念

アイデア：
1. 各ウィンドウを1行に展開（flatten）
2. 全ウィンドウを縦に積み重ねて行列を作成
3. カーネルも1列に展開
4. 行列積で一気に計算

In [ ]:
# im2col の図解

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# 入力画像
sample_img = np.arange(9).reshape(3, 3)
kernel = np.array([[1, 2], [3, 4]])

ax1 = axes[0]
ax1.imshow(sample_img, cmap='Blues')
for i in range(3):
    for j in range(3):
        ax1.text(j, i, f'{sample_img[i,j]}', ha='center', va='center', fontsize=14)
ax1.set_title('入力画像 (3×3)', fontsize=11)
ax1.axis('off')

# im2col 変換後
windows = sliding_window_view(sample_img, window_shape=(2, 2))
im2col_matrix = windows.reshape(-1, 4)  # (出力位置数, カーネルサイズ)

ax2 = axes[1]
ax2.imshow(im2col_matrix, cmap='Blues', aspect='auto')
for i in range(im2col_matrix.shape[0]):
    for j in range(im2col_matrix.shape[1]):
        ax2.text(j, i, f'{im2col_matrix[i,j]:.0f}', ha='center', va='center', fontsize=12)
ax2.set_title('im2col 行列\n(各行=1つのウィンドウ)', fontsize=11)
ax2.set_xlabel('カーネル要素')
ax2.set_ylabel('出力位置')

# カーネルを列ベクトルに
kernel_col = kernel.flatten().reshape(-1, 1)

ax3 = axes[2]
ax3.imshow(kernel_col, cmap='Oranges', aspect='auto')
for i in range(kernel_col.shape[0]):
    ax3.text(0, i, f'{kernel_col[i,0]:.0f}', ha='center', va='center', fontsize=12)
ax3.set_title('カーネル\n(列ベクトル)', fontsize=11)
ax3.set_xticks([])

# 行列積の結果
result = im2col_matrix @ kernel_col
result_2d = result.reshape(2, 2)

ax4 = axes[3]
ax4.imshow(result_2d, cmap='Greens')
for i in range(2):
    for j in range(2):
        ax4.text(j, i, f'{result_2d[i,j]:.0f}', ha='center', va='center', fontsize=14)
ax4.set_title('出力 (行列積の結果)', fontsize=11)
ax4.axis('off')

plt.suptitle('im2col: 畳み込み → 行列積への変換', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n計算過程:")
print(f"  im2col行列: shape {im2col_matrix.shape}")
print(f"  カーネル列: shape {kernel_col.shape}")
print(f"  行列積:     shape {result.shape} → reshape → {result_2d.shape}")

### 4.2 im2col の実装

In [ ]:
def im2col(image, kernel_size, padding=0, stride=1):
    """
    画像をim2col形式の行列に変換
    
    Parameters:
    -----------
    image : ndarray, shape (H, W)
        入力画像
    kernel_size : tuple (kH, kW)
        カーネルサイズ
    padding : int
        パディング幅
    stride : int
        ストライド
    
    Returns:
    --------
    col : ndarray, shape (出力位置数, kH*kW)
    out_shape : tuple (out_H, out_W)
    """
    # パディング
    if padding > 0:
        image = np.pad(image, padding, mode='constant', constant_values=0)
    
    k_h, k_w = kernel_size
    img_h, img_w = image.shape
    
    # 出力サイズ
    out_h = (img_h - k_h) // stride + 1
    out_w = (img_w - k_w) // stride + 1
    
    # sliding_window_view でウィンドウを取得
    windows = sliding_window_view(image, window_shape=(k_h, k_w))
    
    # ストライド適用
    if stride > 1:
        windows = windows[::stride, ::stride]
    
    # (out_h, out_w, k_h, k_w) → (out_h * out_w, k_h * k_w)
    col = windows.reshape(out_h * out_w, k_h * k_w)
    
    return col, (out_h, out_w)


def conv2d_im2col(image, kernel, padding=0, stride=1):
    """
    im2col を使った2D畳み込み
    """
    k_h, k_w = kernel.shape
    
    # im2col 変換
    col, (out_h, out_w) = im2col(image, (k_h, k_w), padding, stride)
    
    # カーネルを列ベクトルに
    kernel_col = kernel.flatten()
    
    # 行列積
    output = col @ kernel_col
    
    # 2次元に変形
    output = output.reshape(out_h, out_w)
    
    return output

In [ ]:
# im2col の動作確認

test_img = np.random.rand(128, 128)
test_kernel = np.random.rand(3, 3)

result_im2col = conv2d_im2col(test_img, test_kernel, padding=1)
result_naive = conv2d_naive(test_img, test_kernel, padding=1)

print(f"im2col版と愚直版の結果の差: {np.max(np.abs(result_im2col - result_naive)):.2e}")

# 速度測定
start = time.time()
for _ in range(10):
    _ = conv2d_im2col(test_img, test_kernel, padding=1)
im2col_time = (time.time() - start) / 10

print(f"\nim2col版の計算時間: {im2col_time*1000:.2f} ms")

### 4.3 GPUが行列積を得意とする理由

im2col が重要なのは、**GPUが行列積を非常に高速に計算できる**からです。

In [ ]:
# 行列積の並列性を図解

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左：畳み込みの計算パターン（依存関係が複雑）
ax1 = axes[0]
ax1.axis('off')
conv_text = """
【畳み込み（愚直版）の計算パターン】

for i in range(out_h):
    for j in range(out_w):
        window = image[i:i+k, j:j+k]
        output[i,j] = sum(window * kernel)

問題点:
• ループの各イテレーションが独立だが
  Pythonでは順次実行される
• メモリアクセスパターンが複雑
• キャッシュ効率が悪い
"""
ax1.text(0.1, 0.5, conv_text, fontsize=11, family='monospace',
         verticalalignment='center', transform=ax1.transAxes,
         bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))
ax1.set_title('畳み込み（ループ版）', fontsize=12)

# 右：行列積の計算パターン（高度に並列化可能）
ax2 = axes[1]
ax2.axis('off')
gemm_text = """
【行列積（im2col版）の計算パターン】

col = im2col(image)  # 一度の変換
output = col @ kernel_flat  # 高度に最適化された行列積

利点:
• BLAS/cuBLAS が超高速に計算
  - SIMD命令（CPU）
  - 数千コアの並列実行（GPU）
• メモリアクセスが連続的
• キャッシュ効率が良い

GPU性能: 
  行列積は FLOPS の理論値に近い速度を達成
"""
ax2.text(0.1, 0.5, gemm_text, fontsize=11, family='monospace',
         verticalalignment='center', transform=ax2.transAxes,
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
ax2.set_title('行列積（im2col版）', fontsize=12)

plt.suptitle('なぜ im2col が高速なのか', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 5. PyTorchの内部実装との比較

PyTorchの `torch.nn.functional.conv2d` と比較してみましょう。

In [ ]:
if TORCH_AVAILABLE:
    # PyTorchでの畳み込み
    def conv2d_torch(image, kernel, padding=0, stride=1):
        """
        PyTorchを使った2D畳み込み
        """
        # NumPy → PyTorch tensor
        # conv2d expects (batch, channel, H, W)
        img_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
        kernel_tensor = torch.from_numpy(kernel).float().unsqueeze(0).unsqueeze(0)
        
        # 畳み込み実行
        output = F.conv2d(img_tensor, kernel_tensor, padding=padding, stride=stride)
        
        # PyTorch tensor → NumPy
        return output.squeeze().numpy()
    
    # 動作確認
    test_img = np.random.rand(128, 128)
    test_kernel = np.random.rand(3, 3)
    
    result_torch = conv2d_torch(test_img, test_kernel, padding=1)
    result_naive = conv2d_naive(test_img, test_kernel, padding=1)
    
    print(f"PyTorch版と愚直版の結果の差: {np.max(np.abs(result_torch - result_naive)):.2e}")
    print("→ 数値誤差の範囲で一致")
else:
    print("PyTorchが利用できないため、このセクションはスキップされます")

In [ ]:
if TORCH_AVAILABLE:
    # PyTorchの速度測定
    
    # ウォームアップ
    for _ in range(5):
        _ = conv2d_torch(test_img, test_kernel, padding=1)
    
    # 測定
    start = time.time()
    for _ in range(100):
        _ = conv2d_torch(test_img, test_kernel, padding=1)
    torch_time = (time.time() - start) / 100
    
    print(f"PyTorch版の計算時間: {torch_time*1000:.3f} ms")
    print(f"\n注: NumPy↔PyTorch変換のオーバーヘッドを含む")
    print(f"    実際のCNN訓練では、データは常にGPU上にあるため")
    print(f"    この変換コストはかからない")

---

## 6. ベンチマーク：3つの実装の速度比較

全ての実装を様々な条件で比較します。

In [ ]:
def benchmark_convolutions(sizes, kernel_size=3, n_runs=5):
    """
    様々なサイズで畳み込み実装をベンチマーク
    """
    results = {
        'size': [],
        'naive': [],
        'vectorized': [],
        'im2col': [],
    }
    
    if TORCH_AVAILABLE:
        results['pytorch'] = []
    
    kernel = np.random.rand(kernel_size, kernel_size)
    
    for size in sizes:
        print(f"Testing size {size}×{size}...", end=' ')
        img = np.random.rand(size, size)
        results['size'].append(size)
        
        # 愚直版
        if size <= 256:  # 大きいサイズでは遅すぎるのでスキップ
            times = []
            for _ in range(n_runs):
                start = time.time()
                _ = conv2d_naive(img, kernel, padding=1)
                times.append(time.time() - start)
            results['naive'].append(np.mean(times))
        else:
            results['naive'].append(np.nan)
        
        # ベクトル化版
        times = []
        for _ in range(n_runs):
            start = time.time()
            _ = conv2d_vectorized(img, kernel, padding=1)
            times.append(time.time() - start)
        results['vectorized'].append(np.mean(times))
        
        # im2col版
        times = []
        for _ in range(n_runs):
            start = time.time()
            _ = conv2d_im2col(img, kernel, padding=1)
            times.append(time.time() - start)
        results['im2col'].append(np.mean(times))
        
        # PyTorch版
        if TORCH_AVAILABLE:
            times = []
            for _ in range(n_runs):
                start = time.time()
                _ = conv2d_torch(img, kernel, padding=1)
                times.append(time.time() - start)
            results['pytorch'].append(np.mean(times))
        
        print("done")
    
    return results

# ベンチマーク実行
sizes = [32, 64, 128, 256, 512]
print("ベンチマーク開始...\n")
bench_results = benchmark_convolutions(sizes)

In [ ]:
# 結果の可視化

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sizes_arr = np.array(bench_results['size'])

# 左：絶対時間
ax1 = axes[0]
ax1.semilogy(sizes_arr, np.array(bench_results['naive'])*1000, 'o-', label='愚直版', linewidth=2, markersize=8)
ax1.semilogy(sizes_arr, np.array(bench_results['vectorized'])*1000, 's-', label='ベクトル化版', linewidth=2, markersize=8)
ax1.semilogy(sizes_arr, np.array(bench_results['im2col'])*1000, '^-', label='im2col版', linewidth=2, markersize=8)
if TORCH_AVAILABLE:
    ax1.semilogy(sizes_arr, np.array(bench_results['pytorch'])*1000, 'd-', label='PyTorch', linewidth=2, markersize=8)
ax1.set_xlabel('画像サイズ (pixels)', fontsize=11)
ax1.set_ylabel('計算時間 (ms)', fontsize=11)
ax1.set_title('絶対計算時間（対数スケール）', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 右：愚直版に対する高速化率
ax2 = axes[1]
naive_times = np.array(bench_results['naive'])
valid_idx = ~np.isnan(naive_times)

speedup_vec = naive_times[valid_idx] / np.array(bench_results['vectorized'])[valid_idx]
speedup_im2col = naive_times[valid_idx] / np.array(bench_results['im2col'])[valid_idx]

ax2.bar(np.arange(sum(valid_idx)) - 0.2, speedup_vec, width=0.4, label='ベクトル化版')
ax2.bar(np.arange(sum(valid_idx)) + 0.2, speedup_im2col, width=0.4, label='im2col版')
ax2.set_xticks(np.arange(sum(valid_idx)))
ax2.set_xticklabels(sizes_arr[valid_idx])
ax2.set_xlabel('画像サイズ (pixels)', fontsize=11)
ax2.set_ylabel('高速化率 (倍)', fontsize=11)
ax2.set_title('愚直版に対する高速化率', fontsize=12)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.suptitle('畳み込み実装のベンチマーク（3×3カーネル）', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 結果の表形式出力

print("\n" + "="*80)
print("ベンチマーク結果まとめ")
print("="*80)

header = f"{'サイズ':>8} {'愚直版(ms)':>12} {'ベクトル化(ms)':>14} {'im2col(ms)':>12}"
if TORCH_AVAILABLE:
    header += f" {'PyTorch(ms)':>12}"
print(header)
print("-"*80)

for i, size in enumerate(bench_results['size']):
    naive_t = bench_results['naive'][i]
    vec_t = bench_results['vectorized'][i]
    im2col_t = bench_results['im2col'][i]
    
    naive_str = f"{naive_t*1000:.2f}" if not np.isnan(naive_t) else "N/A"
    row = f"{size:>8} {naive_str:>12} {vec_t*1000:>14.2f} {im2col_t*1000:>12.2f}"
    
    if TORCH_AVAILABLE:
        torch_t = bench_results['pytorch'][i]
        row += f" {torch_t*1000:>12.3f}"
    
    print(row)

print("="*80)

---

## まとめ

In [ ]:
# まとめの図

summary = """
╔═══════════════════════════════════════════════════════════════════════════════╗
║                    畳み込み高速化のまとめ                                     ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【3つの実装アプローチ】                                                      ║
║                                                                               ║
║  1. 愚直版（forループ）                                                       ║
║     + 理解しやすい                                                            ║
║     - 非常に遅い（Pythonループのオーバーヘッド）                              ║
║                                                                               ║
║  2. ベクトル化版（sliding_window_view）                                       ║
║     + メモリ効率が良い（コピーなし）                                          ║
║     + 実装がシンプル                                                          ║
║     + 愚直版の10-100倍高速                                                    ║
║                                                                               ║
║  3. im2col版                                                                  ║
║     + 行列積に変換（BLAS/cuBLASで超高速）                                     ║
║     + GPU実装との親和性が高い                                                 ║
║     - メモリ使用量が増加                                                      ║
║                                                                               ║
╠═══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  【実際のCNNフレームワークでは】                                              ║
║                                                                               ║
║  • PyTorch, TensorFlow: cuDNN（NVIDIAの高度最適化ライブラリ）を使用          ║
║  • im2col の改良版や、Winograd変換など複数のアルゴリズムを状況に応じて選択   ║
║  • GPU上では、愚直版の1000倍以上の速度を達成                                  ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
"""

print(summary)

### このノートブックで学んだこと

1. **sliding_window_view の使い方**
   - メモリをコピーせずに全ウィンドウを取得
   - 4次元配列 (out_h, out_w, k_h, k_w) として扱える

2. **ベクトル化畳み込み**
   - `np.sum(windows * kernel, axis=(-2, -1))` で一発計算
   - forループ版の10-100倍高速

3. **im2col テクニック**
   - 畳み込み → 行列積への変換
   - GPUでの高速計算の基盤

4. **高速化の本質**
   - Pythonループを排除
   - ベクトル演算・行列演算に変換
   - 最適化されたBLASライブラリを活用

---

## 次のステップ

次のノートブック **84. 古典的フィルタの解剖学** では：

- 平滑化フィルタ（ボックス、ガウシアン）
- 微分フィルタ（Prewitt、Sobel、Laplacian）
- 複合フィルタ（シャープ化、LoG、DoG）
- 周波数領域での解釈（FFT）

自分で設計したカーネルを使って、画像処理の古典的手法を実践します。